<a href="https://colab.research.google.com/github/dgavieira/autoencoders-recommendation/blob/main/trabalho_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

movies_url = "/content/drive/MyDrive/Mestrado/PGENE613-Topicos-Avancados-Aprendizado-Maquina-Otimizacao/trabalho-2/Movies/movies.csv"
ratings_url = "/content/drive/MyDrive/Mestrado/PGENE613-Topicos-Avancados-Aprendizado-Maquina-Otimizacao/trabalho-2/Movies/ratings.csv"

movies = pd.read_csv(movies_url)
ratings = pd.read_csv(ratings_url)

# Exibindo as 5 primeiras linhas de cada DataFrame
print("Filmes:")
print(movies.head())
print("\nAvaliações:")
print(ratings.head())

# Preparar a matriz de ratings (usuários x filmes)
n_users = ratings['userId'].nunique()
n_movies = len(movies)  # Usar o número total de filmes no arquivo movies.csv

# Criando uma matriz de usuários e filmes, preenchendo com as avaliações
Y = np.zeros((n_users, n_movies))

# Mapeamento de IDs de filmes e usuários para índices consecutivos
movie_indices = {movie_id: idx for idx, movie_id in enumerate(movies['movieId'])}
user_indices = {user_id: idx for idx, user_id in enumerate(ratings['userId'].unique())}

# Preencher a matriz Y com as avaliações
for row in ratings.itertuples():
    user_idx = user_indices[row.userId]
    if row.movieId in movie_indices:  # Certificar que o filme existe na lista de filmes
        movie_idx = movie_indices[row.movieId]
        Y[user_idx, movie_idx] = row.rating

# Exibir dimensões da matriz de ratings Y
print(f'\nDimensões da matriz de avaliações Y: {Y.shape}')

Filmes:
   movieId                               title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride Part II (1995)   

                                        genres  
0  Adventure|Animation|Children|Comedy|Fantasy  
1                   Adventure|Children|Fantasy  
2                               Comedy|Romance  
3                         Comedy|Drama|Romance  
4                                       Comedy  

Avaliações:
   userId  movieId  rating  timestamp
0       1        1     4.0  964982703
1       1        3     4.0  964981247
2       1        6     4.0  964982224
3       1       47     5.0  964983815
4       1       50     5.0  964982931

Dimensões da matriz de avaliações Y: (610, 9742)


In [ ]:
# Função para calcular o erro médio quadrático
def mse(Y, R):
    return np.mean((Y - R) ** 2)

# Inicializar aleatoriamente as matrizes W e V
def initialize_factors(n_users, n_movies, k):
    W = np.random.rand(n_users, k)  # Fatores de usuário
    V = np.random.rand(n_movies, k)  # Fatores de filme
    return W, V

In [ ]:
# Treinamento com Fatoração de Matriz
def train_matrix_factorization(Y, k, alpha, lambda_, max_iters, threshold):
    n_users, n_movies = Y.shape
    W, V = initialize_factors(n_users, n_movies, k)

    # Conjunto S = {(i,j); (entrada i ≠ 0, saída j ≠ 0)}
    S = [(i, j) for i in range(n_users) for j in range(n_movies) if Y[i, j] > 0]

    for iteration in range(max_iters):
        np.random.shuffle(S)  # Misturar aleatoriamente os dados

        for i, j in S:
            # Cálculo do erro eij
            eij = Y[i, j] - np.dot(W[i, :], V[j, :])

            # Atualizar W e V
            W[i, :] = W[i, :] * (1 - 2 * alpha * lambda_) + 2 * alpha * eij * V[j, :]
            V[j, :] = V[j, :] * (1 - 2 * alpha * lambda_) + 2 * alpha * eij * W[i, :]

        # Verificar condição de convergência
        R = np.dot(W, V.T)
        current_mse = mse(Y, R)
        print(f'Iteração {iteration+1}: MSE = {current_mse}')

        if current_mse < threshold:
            print(f'Convergência alcançada com MSE = {current_mse}')
            break

    return W, V, current_mse

In [ ]:
# Parâmetros do modelo
alpha = 0.005  # Taxa de aprendizado
lambda_ = 0.0005  # Fator de regularização
k = 50  # Número de neurônios escondidos
max_iters = 100  # Número máximo de iterações
threshold = 0.1  # Condição de convergência (MSE)

# Exemplo: Simulação dos dados de entrada Y (substituir pela matriz real)
# Aqui, Y seria a matriz de avaliações de filmes pelos usuários
n_users, n_movies = 610, 9742  # Exemplo: número de usuários e filmes
Y = np.random.rand(n_users, n_movies)  # Simulação: substitua pelos dados reais

# Treinamento do modelo
W, V, final_mse = train_matrix_factorization(Y, k, alpha, lambda_, max_iters, threshold)

# Matriz de predições final
R = np.dot(W, V.T)

# Exibindo o erro final e a matriz de predições
print(f"Erro final (MSE): {final_mse}")
print(f"Matriz de predições (R):\n {R}")


Iteração 1: MSE = 0.09152372839494378
Convergência alcançada com MSE = 0.09152372839494378
Erro final (MSE): 0.09152372839494378
Matriz de predições (R):
 [[0.43887285 0.47810525 0.45458883 ... 0.5436856  0.33054036 0.55024043]
 [0.43265002 0.58795245 0.50758611 ... 0.47446109 0.41389703 0.53335908]
 [0.54240172 0.53896848 0.56663684 ... 0.61627154 0.51553623 0.55893564]
 ...
 [0.48103732 0.4513035  0.44068525 ... 0.38328378 0.38827846 0.51688026]
 [0.34702749 0.48351116 0.41943084 ... 0.44250675 0.36676215 0.40111707]
 [0.30904719 0.53421814 0.4022599  ... 0.4247141  0.44727329 0.43874286]]


In [ ]:
# Importar as bibliotecas necessárias
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Carregar os dados
movies_url = "/content/drive/MyDrive/Mestrado/PGENE613-Topicos-Avancados-Aprendizado-Maquina-Otimizacao/trabalho-2/Movies/movies.csv"
ratings_url = "/content/drive/MyDrive/Mestrado/PGENE613-Topicos-Avancados-Aprendizado-Maquina-Otimizacao/trabalho-2/Movies/ratings.csv"

movies = pd.read_csv(movies_url)
ratings = pd.read_csv(ratings_url)

# Preparar a matriz de ratings (usuários x filmes)
n_users = ratings['userId'].nunique()
n_movies = len(movies)

# Criar a matriz de usuários e filmes, preenchendo com as avaliações
Y = np.zeros((n_users, n_movies))

# Mapeamento de IDs de filmes e usuários para índices consecutivos
movie_indices = {movie_id: idx for idx, movie_id in enumerate(movies['movieId'])}
user_indices = {user_id: idx for idx, user_id in enumerate(ratings['userId'].unique())}

# Preencher a matriz Y com as avaliações
for row in ratings.itertuples():
    user_idx = user_indices[row.userId]
    if row.movieId in movie_indices:
        movie_idx = movie_indices[row.movieId]
        Y[user_idx, movie_idx] = row.rating

# Dividir os dados em treino e teste
Y_train, Y_test = train_test_split(Y, test_size=0.2, random_state=42)

# Parâmetros do modelo
k_values = [50, 75, 100]  # Três valores de k sugeridos
alpha = 0.005  # Taxa de aprendizado
lmbda = 0.0005  # Fator de regularização (10x menor que a taxa de aprendizado)

# Definir e treinar o autoencoder
def train_autoencoder(Y_train, k, alpha, lmbda, epochs=100, batch_size=32):
    input_dim = Y_train.shape[1]
    # Definir o modelo de autoencoder
    input_layer = tf.keras.layers.Input(shape=(input_dim,))
    hidden_layer = tf.keras.layers.Dense(k, activation='relu',
                                         kernel_regularizer=tf.keras.regularizers.l2(lmbda))(input_layer)
    output_layer = tf.keras.layers.Dense(input_dim, activation='linear')(hidden_layer)

    model = tf.keras.Model(inputs=input_layer, outputs=output_layer)
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=alpha), loss='mse')

    # Treinar o modelo
    model.fit(Y_train, Y_train, epochs=epochs, batch_size=batch_size, validation_split=0.1, verbose=1)

    return model

# Treinando o modelo com k = 75 como exemplo
k = 75
autoencoder = train_autoencoder(Y_train, k, alpha, lmbda)

# Fazer previsões para os dados de teste
Y_pred = autoencoder.predict(Y)

# Gerar recomendações para usuários específicos
user_ids = [40, 92, 123, 245, 312, 460, 514, 590]

def get_recommendations(user_id, Y_pred, Y, movie_indices, movies, n_recommendations=5):
    user_idx = user_indices[user_id]

    # Obter as previsões de ratings do usuário
    user_ratings_pred = Y_pred[user_idx]

    # Remover os filmes já avaliados
    already_rated_indices = np.where(Y[user_idx] > 0)[0]
    user_ratings_pred[already_rated_indices] = -1  # Definir como -1 para ignorar

    # Selecionar os melhores filmes recomendados
    top_movie_indices = np.argsort(user_ratings_pred)[-n_recommendations:][::-1]
    top_movies = [movies.iloc[idx]['title'] for idx in top_movie_indices]

    return top_movies

# Gerar as recomendações e calcular o erro MSE para os usuários
recommendations = []
mse_values = []

for user_id in user_ids:
    top_movies = get_recommendations(user_id, Y_pred, Y, movie_indices, movies)
    recommendations.append(top_movies)

    # Calcular o MSE para os filmes que o usuário já avaliou
    user_idx = user_indices[user_id]
    user_ratings_true = Y[user_idx][Y[user_idx] > 0]  # Ratings reais
    user_ratings_pred = Y_pred[user_idx][Y[user_idx] > 0]  # Ratings preditos
    mse = mean_squared_error(user_ratings_true, user_ratings_pred)
    mse_values.append(mse)

# Criar a tabela final
user_ids_str = [str(uid) for uid in user_ids]
columns = ['Filme 1', 'Filme 2', 'Filme 3', 'Filme 4', 'Filme 5', 'MSE']

df_recommendations = pd.DataFrame(
    data=[rec + [mse] for rec, mse in zip(recommendations, mse_values)],
    index=user_ids_str,
    columns=columns
)

# Exibir a tabela final de recomendações
print(df_recommendations)


Epoch 1/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 3s 94ms/step - loss: 0.2227 - val_loss: 0.2088
Epoch 2/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.1847 - val_loss: 0.2253
Epoch 3/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.1855 - val_loss: 0.2249
Epoch 4/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.1941 - val_loss: 0.2264
Epoch 5/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.1799 - val_loss: 0.2162
Epoch 6/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.1808 - val_loss: 0.2261
Epoch 7/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.1796 - val_loss: 0.2132
Epoch 8/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.1728 - val_loss: 0.2123
Epoch 9/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.1979 - val_loss: 0.2202
Epoch 10/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.1916 - val_loss: 0.2328
Epoch 11/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.2161 - val_loss: 0.2560
Epoch 12/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.

In [ ]:
# Importar as bibliotecas necessárias
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.metrics import mean_squared_error

# Carregar os dados
# Carregar os dados
movies_url = "/content/drive/MyDrive/Mestrado/PGENE613-Topicos-Avancados-Aprendizado-Maquina-Otimizacao/trabalho-2/Movies/movies.csv"
ratings_url = "/content/drive/MyDrive/Mestrado/PGENE613-Topicos-Avancados-Aprendizado-Maquina-Otimizacao/trabalho-2/Movies/ratings.csv"

movies = pd.read_csv(movies_url)
ratings = pd.read_csv(ratings_url)

# Preparar a matriz de ratings (usuários x filmes)
n_users = ratings['userId'].nunique()
n_movies = len(movies)

# Criar a matriz de usuários e filmes, preenchendo com as avaliações
Y = np.zeros((n_users, n_movies))

# Mapeamento de IDs de filmes e usuários para índices consecutivos
movie_indices = {movie_id: idx for idx, movie_id in enumerate(movies['movieId'])}
user_indices = {user_id: idx for idx, user_id in enumerate(ratings['userId'].unique())}

# Preencher a matriz Y com as avaliações
for row in ratings.itertuples():
    user_idx = user_indices[row.userId]
    if row.movieId in movie_indices:
        movie_idx = movie_indices[row.movieId]
        Y[user_idx, movie_idx] = row.rating

# Definir o autoencoder
def build_autoencoder(input_dim, k, alpha, lmbda):
    # Definir a arquitetura do autoencoder
    input_layer = tf.keras.layers.Input(shape=(input_dim,))
    hidden_layer = tf.keras.layers.Dense(k, activation='relu',
                                         kernel_initializer='random_uniform',
                                         bias_initializer='random_uniform',
                                         kernel_regularizer=tf.keras.regularizers.l2(lmbda))(input_layer)
    output_layer = tf.keras.layers.Dense(input_dim, activation='linear')(hidden_layer)

    model = tf.keras.Model(inputs=input_layer, outputs=output_layer)
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=alpha), loss='mse')

    return model

# Definir parâmetros
alpha = 0.005  # Taxa de aprendizado
lmbda = 0.0005  # Fator de regularização (10x menor que alpha)
k_values = [50, 75, 100]  # Três valores de k (neurônios na camada escondida)
epochs = 100
batch_size = 32

# Dividir dados em treino e teste (opcional, mas aqui treinamos com todo o dataset)
Y_train = Y

# Função para treinar e testar o autoencoder com diferentes valores de k
def train_and_evaluate_autoencoder(Y_train, k_values, alpha, lmbda, mse_threshold=0.1):
    for k in k_values:
        print(f'Treinando com k = {k}...')
        model = build_autoencoder(input_dim=Y_train.shape[1], k=k, alpha=alpha, lmbda=lmbda)

        # Treinar o modelo
        model.fit(Y_train, Y_train, epochs=epochs, batch_size=batch_size, verbose=1, validation_split=0.1)

        # Fazer previsões
        Y_pred = model.predict(Y_train)

        # Calcular o erro médio quadrático global
        mse = mean_squared_error(Y_train[Y_train > 0], Y_pred[Y_train > 0])
        print(f'MSE com k={k}: {mse}')

        # Verificar se o MSE está abaixo do threshold e encerrar a simulação
        if mse < mse_threshold:
            print(f'MSE alcançado menor que {mse_threshold}, finalizando...')
            return Y_pred, mse

    return Y_pred, mse

# Treinar e avaliar o autoencoder com diferentes valores de k
Y_pred, final_mse = train_and_evaluate_autoencoder(Y_train, k_values, alpha, lmbda)

# Definir os usuários para os quais faremos recomendações
user_ids = [40, 92, 123, 245, 312, 460, 514, 590]

# Função para obter as recomendações
def get_recommendations(user_id, Y_pred, Y, movie_indices, movies, n_recommendations=5):
    user_idx = user_indices[user_id]

    # Obter as previsões de ratings do usuário
    user_ratings_pred = Y_pred[user_idx]

    # Remover os filmes já avaliados
    already_rated_indices = np.where(Y[user_idx] > 0)[0]
    user_ratings_pred[already_rated_indices] = -1  # Definir como -1 para ignorar

    # Selecionar os melhores filmes recomendados
    top_movie_indices = np.argsort(user_ratings_pred)[-n_recommendations:][::-1]
    top_movies = [movies.iloc[idx]['title'] for idx in top_movie_indices]

    return top_movies

# Gerar as recomendações e calcular o erro MSE para os usuários
recommendations = []
mse_values = []

for user_id in user_ids:
    top_movies = get_recommendations(user_id, Y_pred, Y, movie_indices, movies)
    recommendations.append(top_movies)

    # Calcular o MSE para os filmes que o usuário já avaliou
    user_idx = user_indices[user_id]
    user_ratings_true = Y[user_idx][Y[user_idx] > 0]  # Ratings reais
    user_ratings_pred = Y_pred[user_idx][Y[user_idx] > 0]  # Ratings preditos
    mse = mean_squared_error(user_ratings_true, user_ratings_pred)
    mse_values.append(mse)

# Criar a tabela final
user_ids_str = [str(uid) for uid in user_ids]
columns = ['Filme 1', 'Filme 2', 'Filme 3', 'Filme 4', 'Filme 5', 'MSE']

df_recommendations = pd.DataFrame(
    data=[rec + [mse] for rec, mse in zip(recommendations, mse_values)],
    index=user_ids_str,
    columns=columns
)

# Exibir a tabela final de recomendações
df_recommendations


Treinando com k = 50...
Epoch 1/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 3s 70ms/step - loss: 0.3034 - val_loss: 0.2815
Epoch 2/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.1915 - val_loss: 0.2872
Epoch 3/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.2147 - val_loss: 0.2794
Epoch 4/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.2157 - val_loss: 0.2730
Epoch 5/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.1805 - val_loss: 0.2741
Epoch 6/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.2252 - val_loss: 0.2946
Epoch 7/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.1980 - val_loss: 0.2881
Epoch 8/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.2338 - val_loss: 0.3110
Epoch 9/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.2150 - val_loss: 0.2828
Epoch 10/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.1911 - val_loss: 0.2816
Epoch 11/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.1978 - val_loss: 0.2832
Epoch 12/100
18/18 ━━━━━━━━━━━━━━━━━━━

,Filme 1,Filme 2,Filme 3,Filme 4,Filme 5,MSE
40,"Matrix, The (1999)",Forrest Gump (1994),Star Wars: Episode IV - A New Hope (1977),Star Wars: Episode V - The Empire Strikes Back...,Fight Club (1999),23.796117
92,"Shawshank Redemption, The (1994)",Forrest Gump (1994),Pulp Fiction (1994),"Silence of the Lambs, The (1991)",Braveheart (1995),24.781250
123,Forrest Gump (1994),Braveheart (1995),Schindler's List (1993),Toy Story (1995),Apollo 13 (1995),25.491071
245,"Shawshank Redemption, The (1994)",Forrest Gump (1994),Pulp Fiction (1994),"Silence of the Lambs, The (1991)",Braveheart (1995),14.571429
312,"Shawshank Redemption, The (1994)",Pulp Fiction (1994),Fight Club (1999),"Silence of the Lambs, The (1991)",American Beauty (1999),22.914798
460,Pulp Fiction (1994),"Silence of the Lambs, The (1991)",Star Wars: Episode IV - A New Hope (1977),Schindler's List (1993),Braveheart (1995),27.472561
514,"Shawshank Redemption, The (1994)","Godfather, The (1972)",Memento (2000),Gladiator (2000),"Usual Suspects, The (1995)",19.485516
590,"Truman Show, The (1998)",Good Will Hunting (1997),"Monsters, Inc. (2001)","Amelie (Fabuleux destin d'Amélie Poulain, Le) ...",Eternal Sunshine of the Spotless Mind (2004),19.667582
